In [1]:
import pandas as pd
import json
import numpy as np
import pymongo
from pymongo import MongoClient

In [2]:
client = MongoClient('127.0.0.1', 27017)
db = client.aoligei
imdb_collection = db.imdb_movies

In [3]:
movies = pd.read_csv('./Data/raw_data/IMDB/title.basics.tsv', sep='\t')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
movies = movies[movies['titleType']=='movie'].reset_index()

In [5]:
movies = movies.drop(['index', 'titleType', 'endYear'], axis=1)

In [6]:
movies = movies.replace('\\N', np.nan)

In [7]:
movies.head(5)

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0000502,Bohemios,Bohemios,0,1905,100,NaN
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography"
2,tt0000615,Robbery Under Arms,Robbery Under Arms,0,1907,NaN,Drama
3,tt0000630,Hamlet,Amleto,0,1908,NaN,Drama
4,tt0000675,Don Quijote,Don Quijote,0,1908,NaN,Drama


In [8]:
movies['genres'] = movies.genres.str.split(',')

In [9]:
ratings = pd.read_csv('./Data/raw_data/IMDB/title.ratings.tsv', sep='\t')

In [10]:
movies_ratings = movies.merge(ratings, how='left', left_on='tconst', right_on='imdb_id')

In [11]:
movies_ratings = movies_ratings.set_index('tconst')

In [12]:
movies_ratings = movies_ratings.drop(['imdb_id'], axis=1)

In [13]:
principles = pd.read_csv('./Data/raw_data/IMDB/title.principals.tsv', sep='\t', usecols=['tconst','nconst', 'category'])

In [14]:
principles = principles.replace('\\N', np.nan)

In [15]:
names = pd.read_csv('./Data/raw_data/IMDB/name.basics.tsv', sep='\t', usecols=['nconst','primaryName'])

In [16]:
principles_withname = principles.merge(names, on='nconst').drop(['nconst'], axis=1)

In [17]:
directors = principles_withname[principles_withname['category'] == 'director']

In [18]:
directors = directors.set_index('tconst')

In [19]:
helper = movies_ratings.merge(directors, how='left', left_index=True, right_index=True)

In [20]:
# helper['primaryName'] = helper.primaryName.replace(np.nan, '', regex=True)

In [21]:
director_list = helper.groupby('tconst').primaryName.apply(list).reset_index()

In [22]:
director_list = director_list.set_index('tconst')

In [23]:
movies_dir = movies_ratings.merge(director_list, how='left', left_index=True, right_index=True)

In [24]:
movies_dir = movies_dir.rename(columns={"primaryName": "Directors"})

In [25]:
movies_dir.head(5)

,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,Directors
tconst,,,,,,,,,
tt0000502,Bohemios,Bohemios,0,1905,100,NaN,4.5,8.0,[Ricardo de Baños]
tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"[Action, Adventure, Biography]",6.1,622.0,[Charles Tait]
tt0000615,Robbery Under Arms,Robbery Under Arms,0,1907,NaN,[Drama],4.5,14.0,[Charles MacMahon]
tt0000630,Hamlet,Amleto,0,1908,NaN,[Drama],3.2,11.0,[Mario Caserini]
tt0000675,Don Quijote,Don Quijote,0,1908,NaN,[Drama],4.9,9.0,[Narciso Cuyàs]


In [26]:
writers = principles_withname[principles_withname['category'] == 'writer']

In [27]:
writers = writers.set_index('tconst')

In [28]:
helper_w = movies_ratings.merge(writers, how='left', left_index=True, right_index=True)

In [29]:
# helper_w['primaryName'] = helper_w.primaryName.replace(np.nan, '', regex=True)

In [30]:
writer_list = helper_w.groupby('tconst').primaryName.apply(list).reset_index().set_index('tconst')

In [31]:
movies_wri = movies_dir.merge(writer_list, how='left', left_index=True, right_index=True).rename(columns={"primaryName": "Writers"})

In [32]:
movies_wri.head(5)

,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,Directors,Writers
tconst,,,,,,,,,,
tt0000502,Bohemios,Bohemios,0,1905,100,NaN,4.5,8.0,[Ricardo de Baños],"[Miguel de Palacios, Guillermo Perrín]"
tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"[Action, Adventure, Biography]",6.1,622.0,[Charles Tait],[nan]
tt0000615,Robbery Under Arms,Robbery Under Arms,0,1907,NaN,[Drama],4.5,14.0,[Charles MacMahon],[Rolf Boldrewood]
tt0000630,Hamlet,Amleto,0,1908,NaN,[Drama],3.2,11.0,[Mario Caserini],[William Shakespeare]
tt0000675,Don Quijote,Don Quijote,0,1908,NaN,[Drama],4.9,9.0,[Narciso Cuyàs],[Miguel de Cervantes y Saavedra]


In [33]:
cast = principles_withname[(principles_withname['category'] == 'actor') | (principles_withname['category'] == 'actress' )]

In [34]:
cast = cast.set_index('tconst')

In [35]:
helper_c = movies_ratings.merge(cast, how='left', left_index=True, right_index=True)

In [36]:
# helper_c['primaryName'] = helper_c.primaryName.replace(np.nan, '', regex=True)

In [37]:
cast_list = helper_c.groupby('tconst').primaryName.apply(list).reset_index().set_index('tconst')

In [38]:
movies_cast = movies_wri.merge(cast_list, how='left', left_index=True, right_index=True).rename(columns={"primaryName": "Casts"})

In [39]:
movies_cast.head(5)

,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,Directors,Writers,Casts
tconst,,,,,,,,,,,
tt0000502,Bohemios,Bohemios,0,1905,100,NaN,4.5,8.0,[Ricardo de Baños],"[Miguel de Palacios, Guillermo Perrín]","[Antonio del Pozo, El Mochuelo]"
tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"[Action, Adventure, Biography]",6.1,622.0,[Charles Tait],[nan],"[Elizabeth Tait, John Tait, Norman Campbell, B..."
tt0000615,Robbery Under Arms,Robbery Under Arms,0,1907,NaN,[Drama],4.5,14.0,[Charles MacMahon],[Rolf Boldrewood],"[Jim Gerald, George Merriman, Lance Vane, Will..."
tt0000630,Hamlet,Amleto,0,1908,NaN,[Drama],3.2,11.0,[Mario Caserini],[William Shakespeare],[Fernanda Negri Pouget]
tt0000675,Don Quijote,Don Quijote,0,1908,NaN,[Drama],4.9,9.0,[Narciso Cuyàs],[Miguel de Cervantes y Saavedra],[nan]


In [40]:
movies_cast = movies_cast.reset_index().rename(columns={"tconst": "_id"})

In [41]:
movies_cast

,_id,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,Directors,Writers,Casts
0,tt0000502,Bohemios,Bohemios,0,1905,100,NaN,4.5,8.0,[Ricardo de Baños],"[Miguel de Palacios, Guillermo Perrín]","[Antonio del Pozo, El Mochuelo]"
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"[Action, Adventure, Biography]",6.1,622.0,[Charles Tait],[nan],"[Elizabeth Tait, John Tait, Norman Campbell, B..."
2,tt0000615,Robbery Under Arms,Robbery Under Arms,0,1907,NaN,[Drama],4.5,14.0,[Charles MacMahon],[Rolf Boldrewood],"[Jim Gerald, George Merriman, Lance Vane, Will..."
3,tt0000630,Hamlet,Amleto,0,1908,NaN,[Drama],3.2,11.0,[Mario Caserini],[William Shakespeare],[Fernanda Negri Pouget]
4,tt0000675,Don Quijote,Don Quijote,0,1908,NaN,[Drama],4.9,9.0,[Narciso Cuyàs],[Miguel de Cervantes y Saavedra],[nan]
...,...,...,...,...,...,...,...,...,...,...,...,...
571836,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,57,[Documentary],NaN,NaN,"[Angela Gurgel, Ana Célia de Oliveira]",[nan],[Oldair Soares Ammom]
571837,tt9916680,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,100,[Documentary],NaN,NaN,[Luis Ospina],[Diego Rojas],[nan]
571838,tt9916706,Dankyavar Danka,Dankyavar Danka,0,2013,NaN,[Comedy],NaN,NaN,[Kanchan Nayak],[Sudhir Nikam],"[Makarand Anaspure, Anvay Bendre, Suvarna Kale..."
571839,tt9916730,6 Gunn,6 Gunn,0,2017,116,NaN,NaN,NaN,[Kiran Gawade],[nan],"[Sunil Barve, Bhushan Pradhan, Pranav Raorane,..."


In [44]:
temp = '{[{
    "_id" : "tt0002739",
    "primaryTitle" : "Carmen",
    "originalTitle" : "Carmen",
    "isAdult" : 0,
    "startYear" : 1914,
    "averageRating" : 2.2,
    "numVotes" : 5,
    "Directors" : [
        "Giovanni Doria",
        "Augusto Turqui"
    ],
    "Writers" : [
        "Prosper Mérimée",
        "Ludovic Halévy",
        "Henri Meilhac"
    ],
    "Casts" : [
        "Suzy Prim",
        "Andrea Habay",
        "Juan Rovira",
        "Margarita Silva"
    ]
},
{
    "_id" : "tt0002743",
    "primaryTitle" : "El Caín moderno",
    "originalTitle" : "El Caín moderno",
    "isAdult" : 0,
    "startYear" : 1913,
    "runtimeMinutes" : "48",

    "Directors" : [
        "Antonio Cuesta",
        "José María Codina"
    ],

    "Casts" : [
        "Juan Belmonte",
        "Flores",
        "El Gallo",
        "Paco Madrid"
    ]
}]
}'

SyntaxError: EOL while scanning string literal (<ipython-input-44-136cac6cf7c8>, line 1)

In [51]:
x =  '{ "name":"John", "city":"New York"}{ "name":"John", "age":30, "city":"New York"}'

In [52]:
y = json.loads(x)

JSONDecodeError: Extra data: line 1 column 36 (char 35)

In [58]:
pd.read_json('try.json', orient='records', lines=True)

,_id,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,Directors,Writers,Casts,genres,averageRating,numVotes
0,tt0002743,El Caín moderno,El Caín moderno,0,1913,48.0,"[Antonio Cuesta, José María Codina]",[None],"[Juan Belmonte, Flores, El Gallo, Paco Madrid]",NaN,NaN,NaN
1,tt0002767,The Count of Monte Cristo,The Count of Monte Cristo,0,1913,69.0,"[Edwin S. Porter, Joseph A. Golden]","[Alexandre Dumas, Charles Fechter]","[Murdock MacQuarrie, James O'Neill, Nance O'Neil]","[Drama, History]",5.9,38.0
2,tt0002790,De muerte a vida,De muerte a vida,0,1913,NaN,"[Ricardo de Baños, Alberto Marro]",[None],[None],None,NaN,NaN
3,tt0002889,The Funny Regiment,NaN,0,1913,32.0,"[Maurice Tourneur, Joseph Faivre]",[Georges Courteline],"[Henri Gouget, Charles Krauss, Henry Roussel, ...",[Short],NaN,NaN


In [60]:
len(movies_cast._id.unique())

571841

In [ ]:
571841